In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define custom scorer for MAPE
def mape_scorer(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

# Load the dataset
df_raw = pd.read_csv('../Test.csv')

# Convert the 'date_time' column to datetime and sort the dataset
df_raw['date_time'] = pd.to_datetime(df_raw['date_time'])
df_raw.sort_values('date_time', inplace=True)

# Extracting non-numeric columns
non_numeric_cols = ['is_holiday', 'weather_type', 'weather_description']

# Group by 'date_time' and aggregate: mean for numeric columns, mode for non-numeric columns
agg_funcs = {col: 'mean' for col in df_raw.columns if col not in non_numeric_cols}
agg_funcs.update({col: lambda x: x.mode()[0] if not x.mode().empty else np.nan for col in non_numeric_cols})

df_aggregated = df_raw.groupby('date_time').agg(agg_funcs)

# Drop duplicate rows
df_aggregated.drop_duplicates(inplace=True)

# Add 'traffic_volume' column with 0 values before 'is_holiday'
df_aggregated.insert(loc=df_aggregated.columns.get_loc('is_holiday'), column='traffic_volume', value=0)

# Display the modified DataFrame
print(df_aggregated)

                              date_time  air_pollution_index   humidity  \
date_time                                                                 
2017-05-18 00:00:00 2017-05-18 00:00:00           133.000000  60.666667   
2017-05-18 01:00:00 2017-05-18 01:00:00           190.500000  56.000000   
2017-05-18 02:00:00 2017-05-18 02:00:00            98.666667  49.000000   
2017-05-18 03:00:00 2017-05-18 03:00:00           147.000000  60.000000   
2017-05-18 04:00:00 2017-05-18 04:00:00           156.666667  75.000000   
...                                 ...                  ...        ...   
2018-09-30 19:00:00 2018-09-30 19:00:00           176.000000  21.000000   
2018-09-30 20:00:00 2018-09-30 20:00:00           214.000000  95.000000   
2018-09-30 21:00:00 2018-09-30 21:00:00           173.000000  63.000000   
2018-09-30 22:00:00 2018-09-30 22:00:00            21.000000  57.000000   
2018-09-30 23:00:00 2018-09-30 23:00:00           116.000000  70.000000   

                     win

In [6]:
df_aggregated.to_csv('testsql.csv', index=False)